### Creating a Knowledge recommender system

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
RS_df=pd.read_csv(r"C:\Binjus Ds\ml\IMDB_series_data.csv")

In [4]:
RS_df.shape

(2000, 13)

In [5]:
RS_df.head()

,Poster_Link,Series_Title,Runtime_of_Series,Certificate,Runtime_of_Episodes,Genre,IMDB_Rating,Overview,Star1,Star2,Star3,Star4,No_of_Votes
0,https://m.media-amazon.com/images/M/MV5BYTRiND...,Game of Thrones,(2011–2019),A,57 min,"Action, Adventure, Drama",9.3,Nine noble families fight for control over the...,Emilia Clarke,Peter Dinklage,Kit Harington,Lena Headey,1773458
1,https://m.media-amazon.com/images/M/MV5BMjhiMz...,Breaking Bad,(2008–2013),18,49 min,"Crime, Drama, Thriller",9.5,A high school chemistry teacher diagnosed with...,Bryan Cranston,Aaron Paul,Anna Gunn,Betsy Brandt,1468887
2,https://m.media-amazon.com/images/M/MV5BMTc5Zm...,The Walking Dead,(2010– ),18+,44 min,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,Andrew Lincoln,Norman Reedus,Melissa McBride,Danai Gurira,854698
3,https://m.media-amazon.com/images/M/MV5BNDVkYj...,Friends,(1994–2004),13+,22 min,"Comedy, Romance",8.9,Follows the personal and professional lives of...,Jennifer Aniston,Courteney Cox,Lisa Kudrow,Matt LeBlanc,829816
4,https://m.media-amazon.com/images/M/MV5BMjEzMD...,Stranger Things,(2016– ),15,51 min,"Drama, Fantasy, Horror",8.7,"When a young boy disappears, his mother, a pol...",Millie Bobby Brown,Finn Wolfhard,Winona Ryder,David Harbour,824966


In [6]:
RS_df.columns

Index(['Poster_Link', 'Series_Title', 'Runtime_of_Series', 'Certificate',
       'Runtime_of_Episodes', 'Genre', 'IMDB_Rating', 'Overview', 'Star1',
       'Star2', 'Star3', 'Star4', 'No_of_Votes'],
      dtype='object')

#### Creating a data frame with relevant features for the recommendation system

In [7]:
RS_df2=RS_df[["Series_Title","Runtime_of_Series","Certificate","Genre","IMDB_Rating","No_of_Votes"]]

In [8]:
RS_df2.dtypes

Series_Title          object
Runtime_of_Series     object
Certificate           object
Genre                 object
IMDB_Rating          float64
No_of_Votes            int64
dtype: object

In [9]:
RS_df2.head()

,Series_Title,Runtime_of_Series,Certificate,Genre,IMDB_Rating,No_of_Votes
0,Game of Thrones,(2011–2019),A,"Action, Adventure, Drama",9.3,1773458
1,Breaking Bad,(2008–2013),18,"Crime, Drama, Thriller",9.5,1468887
2,The Walking Dead,(2010– ),18+,"Drama, Horror, Thriller",8.2,854698
3,Friends,(1994–2004),13+,"Comedy, Romance",8.9,829816
4,Stranger Things,(2016– ),15,"Drama, Fantasy, Horror",8.7,824966


#### checking values in feature Certificate

In [10]:
RS_df2["Certificate"].value_counts()


Certificate
18           231
16           133
16+           49
15+           45
A             42
13            42
18+           41
U             29
12+           18
7             16
UA            14
PG            13
13+            9
All            6
Not Rated      6
15             4
7+             3
R              2
Name: count, dtype: int64

In [11]:
RS_df2["Certificate"].isnull().sum()

1297

In [12]:
RS_df2["Certificate"].unique()

array(['A', '18', '18+', '13+', '15', 'UA', 'U', '15+', '16+', '12+',
       '13', nan, '16', '7', '7+', 'PG', 'All', 'Not Rated', 'R'],
      dtype=object)

#### treating null values in Certificate

In [13]:
RS_df2["Certificate"].fillna(RS_df2["Certificate"].mode()[0], inplace= True)

In [14]:
RS_df2["Certificate"].isnull().sum()

0

#### checking values in Genre 

In [15]:
RS_df2["Genre"].unique()

array(['Action, Adventure, Drama', 'Crime, Drama, Thriller',
       'Drama, Horror, Thriller', 'Comedy, Romance',
       'Drama, Fantasy, Horror', 'Crime, Drama, Mystery',
       'Adventure, Drama, Fantasy', 'Action, Crime, Drama', 'Drama',
       'Drama, Sci-Fi, Thriller', 'Drama, Mystery, Sci-Fi', 'Comedy',
       'Drama, Mystery', 'Action, Adventure, Crime',
       'Animation, Adventure, Comedy', 'Animation, Comedy',
       'Comedy, Drama', 'Biography, Crime, Drama',
       'Comedy, Drama, Romance', 'Crime, Drama',
       'Action, Adventure, Fantasy', 'Action, Crime, Mystery',
       'Action, Adventure, Sci-Fi', 'Comedy, Crime, Drama',
       'Drama, Mystery, Thriller', 'Drama, Romance',
       'Animation, Action, Adventure', 'Action, Comedy, Crime',
       'Adventure, Drama, Sci-Fi', 'Animation, Crime, Drama',
       'Crime, Drama, Fantasy', 'Drama, Fantasy, Sci-Fi',
       'Drama, Fantasy, Mystery', 'Crime, Drama, Horror', 'Comedy, Crime',
       'Action, Adventure, Biography', 'A

In [16]:
RS_df2["Genre"].isnull().sum()

0

In [17]:
RS_df2["Genre"]

0           Action, Adventure, Drama
1             Crime, Drama, Thriller
2            Drama, Horror, Thriller
3                    Comedy, Romance
4             Drama, Fantasy, Horror
                    ...             
1995    Animation, Action, Adventure
1996        Adventure, Comedy, Drama
1997                         Western
1998           Action, Drama, Sci-Fi
1999              Comedy, Reality-TV
Name: Genre, Length: 2000, dtype: object

- Genre contains more than 1 values seperated by commas in a single string
- 1st we have convert it into a list and then have to seperate each values included in a single row

#### Converting values in Genre into a list

In [18]:
RS_df2["Genre"] = list(map(lambda x: x.split(', '), RS_df2["Genre"]))


In [19]:
RS_df2["Genre"]

0           [Action, Adventure, Drama]
1             [Crime, Drama, Thriller]
2            [Drama, Horror, Thriller]
3                    [Comedy, Romance]
4             [Drama, Fantasy, Horror]
                     ...              
1995    [Animation, Action, Adventure]
1996        [Adventure, Comedy, Drama]
1997                         [Western]
1998           [Action, Drama, Sci-Fi]
1999              [Comedy, Reality-TV]
Name: Genre, Length: 2000, dtype: object

In [55]:
clean_imdb=RS_df2.copy()

In [56]:
clean_imdb.to_csv("clean_imdb.csv")

#### seperating values within a single row

In [20]:
s = RS_df2.apply(lambda x: pd.Series(x['Genre']),axis=1).stack().reset_index(level=1,drop=True)

In [21]:
s

0           Action
0        Adventure
0            Drama
1            Crime
1            Drama
           ...    
1998        Action
1998         Drama
1998        Sci-Fi
1999        Comedy
1999    Reality-TV
Length: 4996, dtype: object

In [22]:
s.name='Genre'

#### joining this series into the data frame replacing already existing Genre

In [23]:
gen_df = RS_df2.drop('Genre',axis=1).join(s)

In [24]:
gen_df

,Series_Title,Runtime_of_Series,Certificate,IMDB_Rating,No_of_Votes,Genre
0,Game of Thrones,(2011–2019),A,9.3,1773458,Action
0,Game of Thrones,(2011–2019),A,9.3,1773458,Adventure
0,Game of Thrones,(2011–2019),A,9.3,1773458,Drama
1,Breaking Bad,(2008–2013),18,9.5,1468887,Crime
1,Breaking Bad,(2008–2013),18,9.5,1468887,Drama
...,...,...,...,...,...,...
1998,The Cheat,(2017– ),18,8.8,5111,Action
1998,The Cheat,(2017– ),18,8.8,5111,Drama
1998,The Cheat,(2017– ),18,8.8,5111,Sci-Fi
1999,Comic Book Men,(2012–2018),18,7.7,5109,Comedy


#### checking values in Runtime_of_Series

In [25]:
RS_df2["Runtime_of_Series"].unique()

array(['(2011–2019)', '(2008–2013)', '(2010– )', '(1994–2004)',
       '(2016– )', '(2010–2017)', '(2007–2019)', '(2006–2021)',
       '(2005–2014)', '(2014– )', '(2004–2010)', '(2005–2017)',
       '(2013–2018)', '(2011– )', '(2013–2020)', '(2005–2013)',
       '(2004–2012)', '(2012–2020)', '(2005–2020)', '(2013– )',
       '(1989– )', '(2015–2018)', '(2015–2017)', '(2009–2020)',
       '(2015–2019)', '(1997– )', '(2015–2021)', '(2011–2020)',
       '(1999– )', '(1999–2007)', '(2019– )', '(2017– )', '(2002–2008)',
       '(2017–2020)', '(2009–2017)', '(2013–2019)', '(2003–2019)',
       '(2008–2014)', '(1989–1998)', '(2005– )', '(2005–2008)',
       '(2002–2003)', '(2003–2015)', '(2006–2007)', '(2001–2010)',
       '(II) (2006–2010)', '(2013–2015)', '(2013–2022)', '(2010–2013)',
       '(2009–2015)', '(2017–2019)', '(2014–2020)', '(1999–2013)',
       '(2014–2019)', '(2011–2018)', '(2007–2015)', '(2011–2021)',
       '(1993–2018)', '(1990–1991)', '(2010–2014)', '(2007–2014)',
       '

In [26]:
RS_df2["Runtime_of_Series"].value_counts()

Runtime_of_Series
(2019– )       79
(2020– )       76
(2018– )       63
(2017– )       44
(2016– )       38
               ..
(1982–1989)     1
(1969–2011)     1
-1982           1
(2008–2015)     1
(1955–1975)     1
Name: count, Length: 514, dtype: int64

- This is messed up data
- Using regular expression we are cleaning it and converting in to a tuple
- then tuple is converted to sting
- then start and end years are put on seperate columns

In [27]:
import re

In [28]:
import re


# Define a function to clean each string
def clean_year_string(string):
    # Regular expression pattern to extract start and end years
    pattern = r'\b(\d{4})–?(\d{4})?\b'

    # Search for start and end years in the string
    match = re.search(pattern, string)

    if match:
        # Extract start and end years
        start_year = int(match.group(1))
        end_year = int(match.group(2)) if match.group(2) else 2024
    else:
        # Handle cases where no match is found
        start_year = 1955
        end_year = 2024

    return start_year, end_year

# Clean each string in the feature
cleaned_features = [clean_year_string(string) for string in gen_df["Runtime_of_Series"]]

print(cleaned_features)


[(2011, 2019), (2011, 2019), (2011, 2019), (2008, 2013), (2008, 2013), (2008, 2013), (2010, 2024), (2010, 2024), (2010, 2024), (1994, 2004), (1994, 2004), (2016, 2024), (2016, 2024), (2016, 2024), (2010, 2017), (2010, 2017), (2010, 2017), (2007, 2019), (2007, 2019), (2006, 2021), (2006, 2021), (2006, 2021), (2005, 2014), (2005, 2014), (2014, 2024), (2014, 2024), (2014, 2024), (2004, 2010), (2004, 2010), (2004, 2010), (2005, 2017), (2005, 2017), (2005, 2017), (2013, 2018), (2011, 2024), (2011, 2024), (2011, 2024), (2013, 2020), (2013, 2020), (2013, 2020), (2016, 2024), (2016, 2024), (2016, 2024), (2005, 2013), (2004, 2012), (2004, 2012), (2012, 2020), (2012, 2020), (2012, 2020), (2005, 2020), (2005, 2020), (2005, 2020), (2013, 2024), (2013, 2024), (2013, 2024), (1989, 2024), (1989, 2024), (2011, 2019), (2011, 2019), (2015, 2018), (2015, 2018), (2015, 2018), (2015, 2017), (2015, 2017), (2015, 2017), (2009, 2020), (2009, 2020), (2009, 2020), (2013, 2024), (2013, 2024), (2015, 2019), (2015

In [29]:
gen_df["Runtime_of_Series"]=cleaned_features

In [30]:
gen_df["Runtime_of_Series"]

0       (2011, 2019)
0       (2011, 2019)
0       (2011, 2019)
1       (2008, 2013)
1       (2008, 2013)
            ...     
1998    (2017, 2024)
1998    (2017, 2024)
1998    (2017, 2024)
1999    (2012, 2018)
1999    (2012, 2018)
Name: Runtime_of_Series, Length: 4996, dtype: object

In [31]:
gen_df

,Series_Title,Runtime_of_Series,Certificate,IMDB_Rating,No_of_Votes,Genre
0,Game of Thrones,"(2011, 2019)",A,9.3,1773458,Action
0,Game of Thrones,"(2011, 2019)",A,9.3,1773458,Adventure
0,Game of Thrones,"(2011, 2019)",A,9.3,1773458,Drama
1,Breaking Bad,"(2008, 2013)",18,9.5,1468887,Crime
1,Breaking Bad,"(2008, 2013)",18,9.5,1468887,Drama
...,...,...,...,...,...,...
1998,The Cheat,"(2017, 2024)",18,8.8,5111,Action
1998,The Cheat,"(2017, 2024)",18,8.8,5111,Drama
1998,The Cheat,"(2017, 2024)",18,8.8,5111,Sci-Fi
1999,Comic Book Men,"(2012, 2018)",18,7.7,5109,Comedy


#### converting tuple in to string and stard and end years are seperated

In [32]:
gen_df["Runtime_of_Series"]=gen_df["Runtime_of_Series"].apply(lambda x:",".join(map(str,x)))

In [33]:
gen_df["Runtime_of_Series"]

0       2011,2019
0       2011,2019
0       2011,2019
1       2008,2013
1       2008,2013
          ...    
1998    2017,2024
1998    2017,2024
1998    2017,2024
1999    2012,2018
1999    2012,2018
Name: Runtime_of_Series, Length: 4996, dtype: object

In [34]:
gen_df['start_year'] = gen_df['Runtime_of_Series'].apply(lambda x: str(x).split(',')[0] if x !=np.nan else np.nan)

In [35]:
gen_df['end_year']=gen_df['Runtime_of_Series'].apply(lambda x: str(x).split(',')[1] if x !=np.nan else np.nan)

In [36]:
gen_df

,Series_Title,Runtime_of_Series,Certificate,IMDB_Rating,No_of_Votes,Genre,start_year,end_year
0,Game of Thrones,"2011,2019",A,9.3,1773458,Action,2011,2019
0,Game of Thrones,"2011,2019",A,9.3,1773458,Adventure,2011,2019
0,Game of Thrones,"2011,2019",A,9.3,1773458,Drama,2011,2019
1,Breaking Bad,"2008,2013",18,9.5,1468887,Crime,2008,2013
1,Breaking Bad,"2008,2013",18,9.5,1468887,Drama,2008,2013
...,...,...,...,...,...,...,...,...
1998,The Cheat,"2017,2024",18,8.8,5111,Action,2017,2024
1998,The Cheat,"2017,2024",18,8.8,5111,Drama,2017,2024
1998,The Cheat,"2017,2024",18,8.8,5111,Sci-Fi,2017,2024
1999,Comic Book Men,"2012,2018",18,7.7,5109,Comedy,2012,2018


In [37]:
gen_df.drop(columns="Runtime_of_Series",inplace=True)

In [38]:
gen_df

,Series_Title,Certificate,IMDB_Rating,No_of_Votes,Genre,start_year,end_year
0,Game of Thrones,A,9.3,1773458,Action,2011,2019
0,Game of Thrones,A,9.3,1773458,Adventure,2011,2019
0,Game of Thrones,A,9.3,1773458,Drama,2011,2019
1,Breaking Bad,18,9.5,1468887,Crime,2008,2013
1,Breaking Bad,18,9.5,1468887,Drama,2008,2013
...,...,...,...,...,...,...,...
1998,The Cheat,18,8.8,5111,Action,2017,2024
1998,The Cheat,18,8.8,5111,Drama,2017,2024
1998,The Cheat,18,8.8,5111,Sci-Fi,2017,2024
1999,Comic Book Men,18,7.7,5109,Comedy,2012,2018


In [39]:
gen_df["start_year"].isnull().sum()

0

In [40]:
gen_df["end_year"].isnull().sum()

0

In [41]:
gen_df["start_year"].dtypes

dtype('O')

In [42]:
#creating a function to convert all NaT values into zero and making it as a integer feature
def convert_int(x):
    try:
        return int(x)
    except :
        return 0

In [43]:
gen_df['start_year'] = gen_df['start_year'].apply(convert_int)

In [44]:
gen_df['end_year'] = gen_df['end_year'].apply(convert_int)

In [45]:
cleaned_data= gen_df.copy()

In [46]:
cleaned_data.to_csv("cleaned_imdb_df.csv")

#### Build chart function

In [47]:
def build_chart(gen_df,percentile=0.08):
    #ask for user preferred genre
    print('Input preferred Genre')
    Genre = input().capitalize()
    #ask for lower limit of start year
    print('Input lowest start year')
    low_start_year = int(input())
    #ask for upper limit of start year
    print('Input highest start year')
    high_start_year = int(input())
    #ask for lower limit of end year
    print('Input lowest end year')
    low_end_year = int(input())
    #ask for lower limit of end year
    print('Input highest end year')
    high_end_year = int(input())
    #creating a new variable which stores the preferred movies.Copy the contents of gen_df
    movies = gen_df.copy()
    #filter data based on filters
    movies = movies[(movies['Genre']==Genre) & (movies['start_year']>=low_start_year) & (movies['end_year']<=low_end_year) & (movies['end_year']<=high_end_year)]
    #compute the values for C and m based on this filtered data
    C = movies['IMDB_Rating'].mean()
    m = movies['No_of_Votes'].quantile(percentile)
    #only consider movies that have higher than m votes.save this in a new df
    q_movies = movies.copy().loc[movies['No_of_Votes']>=m]
    #calculate the score using the IMDB formula
    q_movies['score'] = q_movies.apply(lambda x: (x['No_of_Votes']/(x['No_of_Votes']+m)* x['IMDB_Rating']) + (m/(m+x['No_of_Votes'])*C),axis=1)
    #sort values in descending order of their scores
    q_movies = q_movies.sort_values(by='score',ascending=False)
    return q_movies
    

In [54]:
build_chart(gen_df).head()

Input preferred Genre
drama
Input lowest start year
1960
Input highest start year
2010
Input lowest end year
1970
Input highest end year
2020


,Series_Title,Certificate,IMDB_Rating,No_of_Votes,Genre,start_year,end_year,score
1043,The Prisoner,18,8.4,12406,Drama,1967,1968,8.4


In [49]:
build_chart(gen_df).head()

Input preferred Genre
action
Input lowest start year
1975
Input highest start year
1990
Input lowest end year
1980
Input highest end year
2022


,Series_Title,Certificate,IMDB_Rating,No_of_Votes,Genre,start_year,end_year,score
961,Battlestar Galactica,18,7.2,13890,Action,1978,1979,7.136319
1577,Wonder Woman,18,7.1,7146,Action,1975,1979,7.052411
1482,Starsky and Hutch,18,7.0,7862,Action,1975,1979,7.000000
